In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [3]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [4]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True) [0]
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [16]:
#function to count rows and move tables that have less than 10 rows
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(inputdir+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        print(len(df))
    elif np.where(pd.isnull(df)):
        df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        return
    else:
        row_df = pd.DataFrame ()
        row_df = row_df.append({'file': table, 'length': len(df)}, ignore_index=True)
    #print(row_df)

In [ ]:
row_list = list()
for file in files:
    #print(index)
    input = '../../src/data/data/'+index+'/All/'
    output = '../../src/data/data/'+index
    if not os.path.exists(output):
        os.makedirs(output)
    #for _ in tqdm(count_move_small_tables, [file for file in os.columns_sorted(input) if file.endswith('.json.gz')]), total=len([file for file in os.columns_sortedinput) if file.endswith('.json.gz')]):
     #   pass
    df = pd.read_json(os.path.join(input, '{}'.format(file)), compression='gzip', lines=True)
    if len(df) <= 10:
        df.to_json(os.path.join(output+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))
    elif len(df)>10:
        row_list.extend([table,len(df)])
        print(len([file for file in os.listdir(output) if file.endswith('.json.gz')]))

In [5]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [6]:
files = [file for file in os.listdir(path) if file.endswith('.json.gz')]

In [ ]:
row_df = pd.DataFrame ()
for file in files:
        df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
        if len(df) <= 10:
           df.to_json(os.path.join(path+'/under_10_rows/'+file), compression = 'gzip', lines = True)
        print(len(df))

        row_df = row_df.append({'file': file, 'length': len(df)}, ignore_index=True)
        #print(row_list)
   
          
          #os.remove(os.path.join(output, '{}'.format(file)))

print(file, len(files))
print(row_df)

In [17]:
#files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
#run function for all classes & tables with multiprocessing
#row_df = pd.DataFrame ()
if __name__ == '__main__':
    for index in columns_sorted.filename:
        #print(index)
        #setcol = set(columns_sorted.loc[index]['listofcolumns'])
        #print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows_and_NAs/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        #print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        #new_df = pd.DataFrame ()
        #new_df = new_df.append({'file': row_df['filename'.groupby(index))], 'mean_length': row_df.mean(len(df))}, ignore_index=True)
        p.join()

 32%|███▏      | 1774/5518 [00:04<00:06, 619.22it/s]

104



 33%|███▎      | 1837/5518 [00:04<00:09, 392.31it/s]

4
4
3
3
6


 35%|███▍      | 1906/5518 [00:04<00:07, 451.88it/s]

...



35
8
10

810
55
10
9


9
9
105
7



100%|██████████| 12661/12661 [00:15<00:00, 839.71it/s] 

735439
9

4107

5





4
5574353


9
633810

39
7
99
535

6
63
4

74
10












8
76


63109875
67



34


98




8
383839104333
9
3
6
4

8



3
6
9






985778737








543993
687835

107
5
71058

8

5

56


4



9


6




439


88

4468335610344310410
5
56
3
3573
3

6108
5












884
43
9
766
8
9
79810
5

4
4
4





3
1037
9
376
68
5

104
6
4




8

38776109
34
84



3

76


9
3


7


4






8784

71073

36373



7333
9
3


10
107

63
4




9


4
35


6
4
99533105103865
76
4484
573989410


7331010

10



5






468

46
836

10
6


4
44433463103

6
575
9
6
10
45

3
4





3






66
93












9



9
7




4943833





3879973549749966
5




1073

4375
9
863864

4




44
7
8


5
975

7

10



3
10


3
5
44

8


56683734
943

3
4


33



7



84698
45
4




5
6
3
10
9


38


4410

5499

44

104


44
5
9


54

345997107

6
87


44
65








8



3


9
6574
4463
3


97


777

875565


3

43
95
1048
6




10


3






3736464
7






91084


8444337381034956657
1010

5




3





33
5
4
68
45368


3





100%|██████████| 210/210 [00:06<00:00, 31.67it/s] 


In [18]:
print(row_df)

Empty DataFrame
Columns: []
Index: []


In [19]:
row_df.mean(axis = 1).mean()

nan

In [20]:
num_tables=[]
for index in columns_sorted.filename:
    print(index)
    outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows/'
    print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
    num_tables.append(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
len(num_tables)

Book
0
CreativeWork
3474
Event
0
Hotel
0
LocalBusiness
0
MusicAlbum
0
MusicRecording
6440
Person
0
Place
0
Product
0
Recipe
0
Restaurant
0
TVEpisode
0


13

In [ ]:
test_df = pd.read_json('/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/CreativeWork/under_10_rows/CreativeWork_1daywebs.com_September2020.json.gz', compression='gzip', lines=True)
test_df